# CNN Classifier Training for GradCAM Explanation

This notebook trains VGG16, ResNet50, and InceptionV3 classifiers on the Food11 dataset.
The trained models will be used for GradCAM-based explainability analysis.

## Setup Instructions

### For Kaggle:
1. Add the Food11 dataset to your notebook
2. Update `DATASET_PATH` below to point to the dataset location
3. Enable GPU accelerator (Settings → Accelerator → GPU)

### For Colab:
1. Upload your dataset or mount Google Drive
2. Enable GPU (Runtime → Change runtime type → GPU)
3. Update `DATASET_PATH` accordingly

## 1. Installation and Imports

In [ ]:
# Install required packages (if needed)
# !pip install tensorflow matplotlib scikit-learn

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

## 2. Configuration

In [ ]:
# ============= CONFIGURATION =============
# Update these paths according to your environment

# For Kaggle (example):
# DATASET_PATH = '/kaggle/input/food11-image-dataset/food11'

# For Colab with Google Drive:
# from google.colab import drive
# drive.mount('/content/drive')
# DATASET_PATH = '/content/drive/MyDrive/Datasets/Food'

# For local or custom path:
DATASET_PATH = 'Datasets/Food/training'  # Update this!

# Output directory for models
OUTPUT_DIR = 'models/Food'
LOGS_DIR = 'logs'

# Training parameters
IMG_SIZE = (256, 256)
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.1
INITIAL_EPOCHS = 10
FINE_TUNE_EPOCHS = 10

# Model selection (set to True to train that model)
TRAIN_VGG16 = True
TRAIN_RESNET50 = True
TRAIN_INCEPTIONV3 = True

# =========================================

# Create directories
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(LOGS_DIR, exist_ok=True)

print(f"Dataset path: {DATASET_PATH}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Image size: {IMG_SIZE}")
print(f"Batch size: {BATCH_SIZE}")

## 3. Dataset Preparation

In [ ]:
def create_data_generators(dataset_path, img_size, batch_size, validation_split):
    """
    Create training and validation data generators with augmentation
    """
    # Training data generator with augmentation
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        validation_split=validation_split,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        zoom_range=0.2,
        shear_range=0.2,
        fill_mode='nearest'
    )
    
    # Validation data generator (only rescaling)
    val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        validation_split=validation_split
    )
    
    # Training generator
    train_generator = train_datagen.flow_from_directory(
        dataset_path,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training',
        shuffle=True
    )
    
    # Validation generator
    val_generator = val_datagen.flow_from_directory(
        dataset_path,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation',
        shuffle=False
    )
    
    return train_generator, val_generator

# Create generators
train_gen, val_gen = create_data_generators(
    DATASET_PATH, IMG_SIZE, BATCH_SIZE, VALIDATION_SPLIT
)

num_classes = len(train_gen.class_indices)
print(f"\nNumber of classes: {num_classes}")
print(f"Class labels: {train_gen.class_indices}")
print(f"Training samples: {train_gen.samples}")
print(f"Validation samples: {val_gen.samples}")

In [ ]:
# Visualize sample images
def visualize_samples(generator, num_samples=9):
    """
    Visualize sample images from the generator
    """
    images, labels = next(generator)
    class_names = {v: k for k, v in generator.class_indices.items()}
    
    fig, axes = plt.subplots(3, 3, figsize=(12, 12))
    axes = axes.ravel()
    
    for i in range(min(num_samples, len(images))):
        axes[i].imshow(images[i])
        label_idx = np.argmax(labels[i])
        axes[i].set_title(f"Class: {class_names[label_idx]}")
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

print("Sample training images:")
visualize_samples(train_gen)

## 4. Model Building Function

In [ ]:
def build_model(backbone_name, input_shape, num_classes):
    """
    Build a classification model with specified backbone
    
    Args:
        backbone_name: 'VGG16', 'ResNet50', or 'InceptionV3'
        input_shape: Input image shape (height, width, channels)
        num_classes: Number of output classes
    
    Returns:
        Compiled Keras model
    """
    # Select backbone
    if backbone_name == 'VGG16':
        base_model = tf.keras.applications.VGG16(
            input_shape=input_shape,
            include_top=False,
            weights='imagenet'
        )
    elif backbone_name == 'ResNet50':
        base_model = tf.keras.applications.ResNet50(
            input_shape=input_shape,
            include_top=False,
            weights='imagenet'
        )
    elif backbone_name == 'InceptionV3':
        base_model = tf.keras.applications.InceptionV3(
            input_shape=input_shape,
            include_top=False,
            weights='imagenet'
        )
    else:
        raise ValueError(f"Unknown backbone: {backbone_name}")
    
    # Freeze base model
    base_model.trainable = False
    
    # Build classification head
    inputs = tf.keras.Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dense(512, activation='relu', kernel_initializer='he_normal')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(256, activation='relu', kernel_initializer='he_normal')(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    
    model = tf.keras.Model(inputs, outputs)
    
    return model, base_model

# Test model building
print("Testing model building...")
test_model, test_base = build_model('VGG16', (*IMG_SIZE, 3), num_classes)
print(f"✓ Model built successfully")
print(f"Total parameters: {test_model.count_params():,}")
print(f"Trainable parameters: {sum([tf.size(w).numpy() for w in test_model.trainable_weights]):,}")
del test_model, test_base

## 5. Training Function

In [ ]:
def train_model(model_name, train_gen, val_gen, initial_epochs=10, fine_tune_epochs=10):
    """
    Train a model with two-phase training: frozen base + fine-tuning
    """
    print("\n" + "="*70)
    print(f"Training: {model_name}")
    print("="*70)
    
    # Build model
    model, base_model = build_model(model_name, (*IMG_SIZE, 3), num_classes)
    
    # Create output directory
    model_dir = os.path.join(OUTPUT_DIR, model_name)
    os.makedirs(model_dir, exist_ok=True)
    
    # ========== PHASE 1: Train with frozen base ==========
    print(f"\n{'='*70}")
    print("PHASE 1: Training with frozen base model")
    print(f"{'='*70}")
    
    # Compile model
    if model_name == 'VGG16':
        optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top3_acc')]
    )
    
    # Callbacks for phase 1
    callbacks_phase1 = [
        tf.keras.callbacks.ModelCheckpoint(
            filepath=os.path.join(model_dir, 'phase1_best.h5'),
            monitor='val_accuracy',
            save_best_only=True,
            mode='max',
            verbose=1
        ),n        tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            patience=5,
            restore_best_weights=True,
            verbose=1
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=2,
            min_lr=1e-7,
            verbose=1
        ),
        tf.keras.callbacks.TensorBoard(
            log_dir=os.path.join(LOGS_DIR, model_name, 'phase1'),
            histogram_freq=1
        )
    ]
    
    # Train phase 1
    history1 = model.fit(
        train_gen,
        epochs=initial_epochs,
        validation_data=val_gen,
        callbacks=callbacks_phase1,
        verbose=1
    )
    
    # ========== PHASE 2: Fine-tuning ==========
    print(f"\n{'='*70}")
    print("PHASE 2: Fine-tuning with unfrozen layers")
    print(f"{'='*70}")
    
    # Unfreeze base model (last few layers)
    base_model.trainable = True
    
    # Freeze all but the last 4 layers
    for layer in base_model.layers[:-4]:
        layer.trainable = False
    
    print(f"Unfrozen layers: {sum([1 for layer in base_model.layers if layer.trainable])}")
    
    # Recompile with lower learning rate
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
        loss='categorical_crossentropy',
        metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top3_acc')]
    )
    
    # Callbacks for phase 2
    callbacks_phase2 = [
        tf.keras.callbacks.ModelCheckpoint(
            filepath=os.path.join(model_dir, 'Epoch={epoch:02d}-Loss={val_loss:.2f}-Acc={val_accuracy:.2f}.h5'),
            monitor='val_accuracy',
            save_best_only=True,
            mode='max',
            verbose=1
        ),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            patience=5,
            restore_best_weights=True,
            verbose=1
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=2,
            min_lr=1e-8,
            verbose=1
        ),
        tf.keras.callbacks.TensorBoard(
            log_dir=os.path.join(LOGS_DIR, model_name, 'phase2'),
            histogram_freq=1
        )
    ]
    
    # Train phase 2
    history2 = model.fit(
        train_gen,
        epochs=fine_tune_epochs,
        validation_data=val_gen,
        callbacks=callbacks_phase2,
        verbose=1
    )
    
    # Save final model
    final_path = os.path.join(model_dir, f'{model_name}_final.h5')
    model.save(final_path)
    print(f"\n✓ Final model saved to: {final_path}")
    
    # Evaluate
    print("\nEvaluating model on validation set...")
    results = model.evaluate(val_gen, verbose=0)
    print(f"\nFinal Validation Metrics:")
    print(f"  Loss: {results[0]:.4f}")
    print(f"  Accuracy: {results[1]:.4f}")
    print(f"  Top-3 Accuracy: {results[2]:.4f}")
    
    return model, history1, history2

## 6. Visualization Function

In [ ]:
def plot_training_history(history1, history2, model_name):
    """
    Plot training history for both phases
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Combine histories
    epochs1 = len(history1.history['accuracy'])
    epochs2 = len(history2.history['accuracy'])
    total_epochs = epochs1 + epochs2
    
    # Accuracy - Phase 1
    axes[0, 0].plot(history1.history['accuracy'], label='Train Accuracy', marker='o')
    axes[0, 0].plot(history1.history['val_accuracy'], label='Val Accuracy', marker='s')
    axes[0, 0].axvline(x=epochs1-1, color='r', linestyle='--', label='Phase 1 End')
    axes[0, 0].set_title(f'{model_name} - Phase 1: Accuracy')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Loss - Phase 1
    axes[0, 1].plot(history1.history['loss'], label='Train Loss', marker='o')
    axes[0, 1].plot(history1.history['val_loss'], label='Val Loss', marker='s')
    axes[0, 1].axvline(x=epochs1-1, color='r', linestyle='--', label='Phase 1 End')
    axes[0, 1].set_title(f'{model_name} - Phase 1: Loss')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Accuracy - Phase 2
    axes[1, 0].plot(range(epochs1, epochs1+epochs2), history2.history['accuracy'], 
                    label='Train Accuracy', marker='o')
    axes[1, 0].plot(range(epochs1, epochs1+epochs2), history2.history['val_accuracy'], 
                    label='Val Accuracy', marker='s')
    axes[1, 0].set_title(f'{model_name} - Phase 2: Accuracy')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Accuracy')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Loss - Phase 2
    axes[1, 1].plot(range(epochs1, epochs1+epochs2), history2.history['loss'], 
                    label='Train Loss', marker='o')
    axes[1, 1].plot(range(epochs1, epochs1+epochs2), history2.history['val_loss'], 
                    label='Val Loss', marker='s')
    axes[1, 1].set_title(f'{model_name} - Phase 2: Loss')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Loss')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, model_name, f'{model_name}_training_history.png'), 
                dpi=150, bbox_inches='tight')
    plt.show()

## 7. Train Models

In [ ]:
# Dictionary to store trained models and histories
trained_models = {}

# Train VGG16
if TRAIN_VGG16:
    model_vgg, hist1_vgg, hist2_vgg = train_model(
        'VGG16', train_gen, val_gen, 
        INITIAL_EPOCHS, FINE_TUNE_EPOCHS
    )
    trained_models['VGG16'] = (model_vgg, hist1_vgg, hist2_vgg)
    plot_training_history(hist1_vgg, hist2_vgg, 'VGG16')

In [ ]:
# Train ResNet50
if TRAIN_RESNET50:
    model_resnet, hist1_resnet, hist2_resnet = train_model(
        'ResNet50', train_gen, val_gen, 
        INITIAL_EPOCHS, FINE_TUNE_EPOCHS
    )
    trained_models['ResNet50'] = (model_resnet, hist1_resnet, hist2_resnet)
    plot_training_history(hist1_resnet, hist2_resnet, 'ResNet50')

In [ ]:
# Train InceptionV3
if TRAIN_INCEPTIONV3:
    model_inception, hist1_inception, hist2_inception = train_model(
        'InceptionV3', train_gen, val_gen, 
        INITIAL_EPOCHS, FINE_TUNE_EPOCHS
    )
    trained_models['InceptionV3'] = (model_inception, hist1_inception, hist2_inception)
    plot_training_history(hist1_inception, hist2_inception, 'InceptionV3')

## 8. Model Comparison

In [ ]:
# Compare all trained models
print("\n" + "="*70)
print("MODEL COMPARISON")
print("="*70)

results_comparison = []

for model_name, (model, _, _) in trained_models.items():
    print(f"\nEvaluating {model_name}...")
    results = model.evaluate(val_gen, verbose=0)
    results_comparison.append({
        'Model': model_name,
        'Loss': results[0],
        'Accuracy': results[1],
        'Top-3 Acc': results[2]
    })
    print(f"  Loss: {results[0]:.4f}")
    print(f"  Accuracy: {results[1]:.4f}")
    print(f"  Top-3 Accuracy: {results[2]:.4f}")

# Display comparison table
import pandas as pd
df = pd.DataFrame(results_comparison)
print("\nComparison Table:")
print(df.to_string(index=False))

# Save results to CSV
df.to_csv(os.path.join(OUTPUT_DIR, 'model_comparison.csv'), index=False)
print(f"\n✓ Results saved to {os.path.join(OUTPUT_DIR, 'model_comparison.csv')}")

print("\n" + "="*70)
print("TRAINING COMPLETE")
print("="*70)
print("Trained models are ready for GradCAM analysis!")
print(f"Models saved in: {OUTPUT_DIR}")
print(f"Logs saved in: {LOGS_DIR}")

# Optional: Generate classification reports and confusion matrices
def generate_reports(model, model_name, val_gen):
    """
    Generate detailed classification report and confusion matrix
    """
    # Get predictions
    val_gen.reset()
    predictions = model.predict(val_gen, verbose=0)
    y_pred = np.argmax(predictions, axis=1)
    y_true = val_gen.classes
    class_names = list(val_gen.class_indices.keys())
    
    # Classification report
    report = classification_report(y_true, y_pred, target_names=class_names)
    print(f"\nClassification Report for {model_name}:")
    print(report)
    
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title(f'Confusion Matrix - {model_name}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, model_name, f'{model_name}_confusion_matrix.png'), dpi=150)
    plt.show()

# Generate reports for all models (optional - uncomment if needed)
# for model_name, (model, _, _) in trained_models.items():
#     generate_reports(model, model_name, val_gen)

print("\nNotebook execution completed successfully!")
